In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="ticks")

# Pretty display for notebooks
%matplotlib inline

# Load the Census dataset
data = pd.read_csv("No_Starts.csv")

# Success - Display the first record
display(data.head(n=5))

In [ ]:
# change dtypes
#data['BOOKING MONTH']=data['BOOKING MONTH'].astype('datetime64[ns]')
#data['LOSS MONTH']=data['LOSS MONTH'].astype('datetime64[ns]')
#data['OWNER HIRE DATE']=data['OWNER HIRE DATE'].astype('datetime64[ns]')
#data['SELLER HIRE DATE']=data['SELLER HIRE DATE'].astype('datetime64[ns]')
#data['COMPANY ID']=data['COMPANY ID'].astype('int64')
data['SIC']=data['SIC'].astype('object')

In [ ]:
# more info on the data
print(data.shape)
print(data.dtypes)
print(data.describe(include='all'))

In [ ]:
FEIN = data[['COMPANY ID','CLIENT ID']].copy()
FEIN.drop_duplicates(subset=['CLIENT ID'],inplace=True)
#print(FEIN)
FEIN_counts= pd.value_counts(FEIN['COMPANY ID'].values,sort=False)
FEIN_counts.to_dict()
data['FEINs']=data['COMPANY ID'].map(FEIN_counts)
display(data.head(n=5))

In [ ]:
# Treat null-values
data.drop(['COMPANY ID','CLIENT ID','NET NO START','LOSS MONTH','BOOKING KEY','BOOKING MONTH','SELLER HIRE DATE','OWNER HIRE DATE','Incorporated Date','SIC DIVISION','SIC GROUP','DunsNumber','BANK REGION','BANK MARKET','BANK BRAND','No_of_Stores_Franchise__c','RSD','SELLER NAME','PROJECT OWNER'], inplace=True, axis=1)
print(data.shape)
data_final = data.dropna(axis=0,how='any')
print(data_final.shape)
data_final.dtypes

In [ ]:
#Remove null & exclude strings
#data_sns=data.drop(['PRODUCT CODE','OPP TYPE','SELLER NAME','SELLER TENURE','RSD','ASVP','WEEK OF CLOSE','PRODUCT NAME','LEAD SOURCE','COMPANY EE','PRIOR PROVIDER','BILLING STATE','SIC','PROJECT TYPE','START FRAME','PROJECT OWNER','Lost'], axis=1)
#data_sns=data.select_dtypes(exclude=['object'])
#sns.pairplot(data_sns.dropna())

In [ ]:
# Pre# Split the data into features and target label
lost_raw = data_final['Lost']
features_raw = data_final.drop(['Lost'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_raw, lost_raw, test_size=0.2,random_state=1)

In [ ]:
# Correct to fit on training data only
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['OPP DURATION', 'Count_of_Federal_Taxes__c', 'Count_of_SUI_Taxes__c', 'NET SOLD', 'Client Tenure', 'DaysSinceIncorporated', 'OwnerTenure', 'FEINs']

X_train_nm = pd.DataFrame(data = X_train)
X_train_nm[numerical] = scaler.fit_transform(X_train[numerical])

X_test_nm = pd.DataFrame(data = X_test)
X_test_nm[numerical] = scaler.fit_transform(X_test[numerical])


# Show an example of a record with scaling applied
display(X_train_nm.head(n = 2))
display(X_test_nm.head(n = 2))

In [ ]:
# TODO: One-hot encode the 'features_log_minmax_transform' data using pandas.get_dummies()
X_train_final = pd.get_dummies(X_train_nm)
X_test_final = pd.get_dummies(X_test_nm)

# TODO: Encode the 'income_raw' data to numerical values
y_train_final = y_train.apply(lambda x: 1 if x == "NO" else 0)
y_test_final = y_test.apply(lambda x: 1 if x == "NO" else 0)

# Print the number of features after one-hot encoding
encoded_train = list(X_train_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded_train)))

encoded_test = list(X_test_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded_test)))

# Uncomment the following line to see the encoded feature names
# print encoded

In [ ]:
# PCA to reduce dimesionality
from sklearn.decomposition import PCA
pca = PCA(n_components =30)
train_X = pca.fit_transform(X_train_final)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# X, y = load training data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print("classifier accuracy:", clf.score(X_test, y_test))

#regr = RandomForestRegressor()
#regr.fit(X_train, y_train[:,1])
#print("regressor R^2:", regr.score(X_test, y_test[:,1]))